![Colegio Bourbaki](./Images/Bourbaki.png)

## Procesamiento de Lenguaje Natural

### Contexto

El objetivo de este notebook es hacer una demostración de la creación de chatbots estilo ChatGPT con conocimiento de datos específicos.

En primer lugar, enseñaremos cómo conectar con el API de OpenAI para utilizar GPT-3.5 Turbo, el modelo que alimenta a la versión abierta de ChatGPT, desde código.

Después, veremos cómo podemos introducir material a la base de conocimiento del chatbot, para así obtener respuestas más personalizadas.

### Librerías

In [ ]:
#!pip install openai langchain duckdb unstructured chromadb tiktoken pdfminer.six python-dotenv

In [1]:
# NLP Chatbots
import openai
from langchain.document_loaders.unstructured import UnstructuredFileLoader
from langchain.text_splitter import CharacterTextSplitter
from langchain.embeddings.openai import OpenAIEmbeddings
from langchain.vectorstores import Chroma
from langchain.chains import VectorDBQA
from langchain.chat_models import ChatOpenAI

#Utils
import os
from dotenv import load_dotenv
from pdfminer.high_level import extract_text

In [2]:
import warnings
warnings.filterwarnings("ignore")

Conectamos con el API GPT-3.5 por medio de una llave privada a cada usuario.

In [3]:
load_dotenv() # This method loads the variables from .env into the environment

True

In [4]:
#api_key = os.environ['CHATGPT_API_KEY']
api_key = os.getenv("CHATGPT_API_KEY") # This method loads the variables from .env into the environment with dotenv
if api_key is None:
    raise ValueError("API key not found. Please set the CHATGPT_API_KEY environment variable.")

In [5]:
client = openai.OpenAI(
    api_key=api_key,
)

Una vez hecha la conexión con GPT-3.5 podemos aprovechar las capacidades conversacionales de ChatGPT. 

Desde este punto ya es posible integrar asistentes inteligentes a aplicaciones y sistemas.

In [6]:
prompt = "¡Hola! ¿Por qué el cielo es azul?"

In [7]:
chat_completion = client.chat.completions.create(
    messages=[
        {
            "role": "user",
            "content": prompt,
        }
    ],
    model="gpt-3.5-turbo",
    #model="gpt-4",
)

In [8]:
print(chat_completion.choices[0].message.content)

El color del cielo se debe a un fenómeno llamado dispersión de Rayleigh. La luz del sol está formada por diferentes longitudes de onda, y cuando atraviesa la atmósfera de la Tierra, las partículas en la atmósfera desvían las longitudes de onda más cortas (como el azul y el violeta) más que las longitudes de onda más largas.

Esto hace que el azul se disperse por todo el cielo, lo que nos da esa apariencia azulada. En resumen, el cielo es azul debido a la forma en que la luz del sol interactúa con la atmósfera de la Tierra.


Podemos lograr mejores respuestas del modelo si modificamos el atributo **system**.

In [9]:
sistema = "Eres un asistente de poetas, habilidoso en explicar conceptos complejos de programación creativamente."
usuario = "Compón un poema que explique el concepto de recursión en programación."

chat_completion = client.chat.completions.create(
  model="gpt-3.5-turbo",
  messages=[
    {"role": "system", "content": sistema},
    {"role": "user", "content": usuario}
  ]
)

In [10]:
print(chat_completion.choices[0].message.content)

En el vasto mundo de la creación digital,
la recursión emerge con su peculiaridad,
como un bucle que se envuelve en sí mismo,
un viaje profundo, un misterio sin abismo.

Cual reflejo en un espejo que no tiene fin,
se llama a sí misma, sin detenerse en su frenesí,
una función que se llama a sí misma de nuevo,
hasta llegar al caso base, al final del juego.

Es como un cuento que se repite una y otra vez,
hasta encontrar la respuesta, hasta llegar al no sé,
un proceso iterativo, pero de forma distinta,
un laberinto de códigos que la mente compinta.

Así es la recursión, un arte en programación,
un enigma que despierta la imaginación,
un ciclo infinito en el mundo binario,
un concepto fascinante, un desafío necesario.

Que viva la recursión en la poética de la programación,
un bucle infinito de pura inspiración,
donde la lógica y la creatividad se entrelazan,
en un baile eterno que nunca se deshazca.


### Creación de un asistente especializado

A su vez, podemos aprovechar aún más las capacidades de loss LLM haciendo una especie de fine-tuning. La idea consiste en alimentar al modelos con documentos propios para así lograr respuestas informadas sobre ellos.

Esto es posible en GPT-3.5 a través de los encajes y la generación de una base de datos vectorizada.

Primero, extraemos texto desde archivos pdfs...


... y extramos el texto de un pdf para después guardarlo en .txt.

In [11]:
def extractor_texto(ruta):
    # Assume extract_text is a function defined elsewhere to extract text from the given path
    txt = extract_text(ruta)
    
    # Clean and format the text
    replacements = {
        '\n\n\x0c': ' ',  # Remove specific pattern
        '...': ' ',       # Replace ellipses with space
        '\n': ' ',        # Replace newline characters with space
        '  ': ' ',        # Replace double spaces with single space
        "\f": ' ',        # Remove form feed characters
        "-": ' '          # Replace hyphens with space
    }
    
    # Apply replacements
    for old, new in replacements.items():
        txt = txt.replace(old, new)
    
    # Split into paragraphs and filter based on conditions
    paragraphs = txt.split('\n\n')
    paragraphs = [paragraph.strip() for paragraph in paragraphs if len(paragraph.strip()) > 30]
    
    # Join the cleaned paragraphs
    cleaned_text = '\n'.join(paragraphs)
    
    # Write the cleaned text to a file, appending '.txt' to the original path and using utf-16 encoding
    with open(ruta + '.txt', "w", encoding="utf-16") as archivo:
        archivo.write(cleaned_text)
    
    # Optionally, return the cleaned text if needed
    return cleaned_text


In [12]:
ruta = './Data/Feynman1982_Article_SimulatingPhysicsWithComputers.pdf'

In [13]:
paper = extractor_texto(ruta)

In [14]:
with open("./Data/output.txt", "w") as text_file:
    print(paper, file=text_file)

In [15]:
loader = UnstructuredFileLoader('./Data/output.txt')
documents = loader.load()

text_splitter = CharacterTextSplitter(chunk_size=1000, chunk_overlap=0)
texts = text_splitter.split_documents(documents)

embeddings = OpenAIEmbeddings(openai_api_key = api_key)

db = Chroma.from_documents(texts, embeddings)

Created a chunk of size 1460, which is longer than the specified 1000
Created a chunk of size 1493, which is longer than the specified 1000
Created a chunk of size 1480, which is longer than the specified 1000
Created a chunk of size 1385, which is longer than the specified 1000
Created a chunk of size 1425, which is longer than the specified 1000
Created a chunk of size 1405, which is longer than the specified 1000
Created a chunk of size 1348, which is longer than the specified 1000
Created a chunk of size 1484, which is longer than the specified 1000
Created a chunk of size 1354, which is longer than the specified 1000
Created a chunk of size 1467, which is longer than the specified 1000
Created a chunk of size 1318, which is longer than the specified 1000
Created a chunk of size 1437, which is longer than the specified 1000
Created a chunk of size 1490, which is longer than the specified 1000
Created a chunk of size 1315, which is longer than the specified 1000
Created a chunk of s

En seguida, cargamos el documento a la base de datos.

In [16]:
qa = VectorDBQA.from_chain_type(llm=ChatOpenAI(openai_api_key=api_key), chain_type="stuff", vectorstore=db, k=1)

Este modelo ahora se puede utilizar como un ChatGPT con conocimiento especializado:

In [17]:
query = "What the document is about?"
qa.run(query)

"The document is about simulating physics with computers, specifically discussing the possibilities of using computers to understand physical laws and the potential insights that could be gained from this approach. It also mentions the influence of Ed Fredkin on the author's interest in the subject."

In [18]:
chat_completion = client.chat.completions.create(
    messages=[
        {
            "role": "user",
            "content": query,
        }
    ],
    model="gpt-3.5-turbo",
    #model="gpt-4",
)
print(chat_completion.choices[0].message.content)

I'm sorry, but without more information or context, I am unable to determine what the document is about. Please provide more details or specific keywords from the document so I can assist you better.


Y es posible obtener respuestas muy específicas y personalizadas sobre nuestros documentos:

In [19]:
query = "How photons are polarized?"
qa.run(query)

'Photons can be polarized by passing them through certain materials like calcite crystals that have the ability to filter out photons vibrating in specific directions. When a photon is polarized, it means its oscillations are restricted to a certain plane or direction. This can cause the photon to be split into different beams or paths based on its polarization state.'

In [20]:
chat_completion = client.chat.completions.create(
    messages=[
        {
            "role": "user",
            "content": query,
        }
    ],
    model="gpt-3.5-turbo",
    #model="gpt-4",
)
print(chat_completion.choices[0].message.content)

Photons can have different polarizations depending on the direction of their electric and magnetic fields. A photon is said to be linearly polarized when its electric and magnetic fields oscillate in a specific plane, known as the polarization plane. The polarization direction of a photon can be perpendicular or parallel to the direction of its propagation.

Circularly polarized photons have electric and magnetic fields that rotate in a circular motion, either clockwise or counterclockwise. This type of polarization can be right-handed or left-handed, depending on the direction of rotation.

Photons can also be unpolarized, which means that their electric and magnetic fields oscillate in random planes and directions. In this case, the photon is said to have no specific polarization state.

Photons can interact with various materials and undergo processes such as reflection, refraction, and scattering, which can affect their polarization state. Techniques such as polarizers and waveplat

In [21]:
query = "Expand the concept of negative probabilities"
qa.run(query)

'In traditional probability theory, probabilities are always non-negative numbers between 0 and 1. However, in certain quantum mechanical contexts, the concept of negative probabilities arises. In these situations, the probabilities can be negative, meaning that the likelihood of an event occurring is less than no event at all. This concept challenges the traditional understanding of probabilities and is not easily interpreted in terms of standard probability theory. Negative probabilities can be encountered in specific quantum mechanical equations where the probabilities are not necessarily positive.'

In [22]:
chat_completion = client.chat.completions.create(
    messages=[
        {
            "role": "user",
            "content": query,
        }
    ],
    model="gpt-3.5-turbo",
    #model="gpt-4",
)
print(chat_completion.choices[0].message.content)

Negative probabilities are a theoretical concept that have been proposed in the field of mathematics and theoretical physics. They represent a mathematical quantity that is less than zero, which goes against the commonly accepted notion of probabilities being non-negative values between 0 and 1.

One interpretation of negative probabilities is that they can arise when dealing with complex systems that exhibit quantum mechanical behavior. In quantum mechanics, probabilities are typically described using complex numbers, which can give rise to negative values when squared. These negative probabilities are often used in mathematical formulations to describe phenomena such as quantum entanglement or tunneling.

Negative probabilities can also be seen as a way to account for uncertainty and information that is not fully known or understood. In some cases, negative probabilities may be used to represent the lack of information or the presence of hidden variables that are influencing the outc

In [23]:
query = "What are quantum computers?"
qa.run(query)

'Quantum computers are a type of computing device that utilize quantum mechanical phenomena, such as superposition and entanglement, to perform operations on data. Unlike classical computers that use binary bits, quantum computers use quantum bits or qubits. These qubits can exist in multiple states at the same time, allowing quantum computers to perform certain calculations much faster than classical computers for specific types of problems.'

In [24]:
chat_completion = client.chat.completions.create(
    messages=[
        {
            "role": "user",
            "content": query,
        }
    ],
    model="gpt-3.5-turbo",
    #model="gpt-4",
)
print(chat_completion.choices[0].message.content)

Quantum computers are a type of computing device that harnesses the laws of quantum mechanics to process and store information in a fundamentally different way compared to classical computers. Unlike classical bits, which can be either a 0 or a 1, quantum bits (qubits) can exist in a superposition of states, enabling them to represent both 0 and 1 simultaneously.

This allows quantum computers to perform certain types of computations much faster than classical computers, especially for problems involving large amounts of data or complex calculations. Quantum computers have the potential to revolutionize fields such as cryptography, drug discovery, materials science, and optimization problems. However, they are still in the early stages of development and are currently only available in specialized research labs.


**Conexión con la API de OpenAI:**

1) ¿Qué información necesitas para autenticarte y realizar peticiones al API de OpenAI?
2) ¿Cuál es el propósito de utilizar una clave API en la conexión con OpenAI y cómo se debe proteger?
3) ¿Qué diferencias hay entre los distintos modelos de OpenAI y cómo elegirías uno para tu aplicación específica?

**Uso de GPT-3.5 Turbo:**

4) ¿Qué ventajas ofrece GPT-3.5 Turbo para la creación de chatbots comparado con versiones anteriores?
5) ¿Cómo se formulan las peticiones al modelo GPT-3.5 Turbo para generar respuestas coherentes y relevantes?
6) ¿Qué limitaciones tiene el modelo GPT-3.5 Turbo y cómo puedes mitigarlas?

**Introducción de Material a la Base de Conocimiento:**

7) ¿Cómo puedes personalizar las respuestas de GPT-3.5 Turbo utilizando información específica?
8) ¿Cuál es la importancia de la relevancia y precisión del material que se introduce en la base de conocimientos del bot?
9) ¿Qué estrategias se pueden utilizar para mantener actualizada la base de conocimientos del chatbot?

**Personalización y Respuestas del Chatbot:**

10) ¿De qué manera se puede ajustar el tono o el estilo de las respuestas que genera GPT-3.5 Turbo?
11) ¿Cómo afecta el contexto proporcionado a las respuestas generadas por el chatbot?
    Describe un método para evaluar la precisión y utilidad de las respuestas del chatbot.

**Problemas Éticos y de Privacidad:**

12) ¿Cuáles son las consideraciones éticas al utilizar modelos de lenguaje generativos como GPT-3.5 Turbo en un chatbot?
13) ¿Cómo debería manejar un chatbot las solicitudes de datos personales o sensibles de los usuarios?
14) ¿Qué medidas se pueden tomar para garantizar la privacidad y la seguridad de los usuarios al interactuar con un chatbot?

![Lenguaje Matemático](./Images/Matematicas.png)

![Contacto](./Images/Contacto.png)